In [1]:
import rospy
from sensor_msgs.msg import JointState
import pandas as pd
import klampt
from klampt import WorldModel
from klampt.model import ik
from klampt import vis
from allegro_control_demo import HandController
import numpy as np
from threading import Thread,Lock
import time


In [2]:
class HandController:
    def __init__(self):
        rospy.init_node('aaaaa',anonymous = True)
        
        ###GAINS
        self.k_p = np.array([500, 800, 900, 500,500, 800, 900, 500,500, 800, 900, 500,1000, 700, 600, 600])
        self.k_d = np.array([25, 50, 55, 40,25, 50, 55, 40,25, 50, 55, 40,50, 50, 50, 40])
        self.k_i = np.array(16*[10])
        
        
        ### LIMITS :
        self.pos_min = np.array(16*[0])
        self.pos_max = np.array(16*[np.pi*(90/180)])
        self.i_saturation = np.array(16*[0.5])
        self.error_integral = np.zeros(16)
        
        ### CONSTANTS
        self.conversion_constant = 1/2400.0
        self.joint_state_subscriber = rospy.Subscriber('/allegroHand_0/joint_states',JointState,self.update_internal_variables,tcp_nodelay = True)
        self.joint_cmd_publisher = test_pub = rospy.Publisher('/allegroHand_0/joint_cmd', JointState, queue_size=10,tcp_nodelay = True)
        self.read_lock = Lock()
        self.target_lock = Lock()
        self.command_lock = Lock()
        self.pos_target = np.array([-0.034044832868002486, 0.21487189444754157, 0.7170365380274353, 0.6094390445703134, -0.006500337101204626, 0.18626165632739455, 0.7353485653664781, 0.7596449737913296, 0.0639623637387915, 0.31196840425314604, 0.7560199640338172, 0.6387554320699687, 1.1008734155170343, 0.7052834691730496, 0.3054462763105439, 0.8326928125510552])

        self.r = rospy.Rate(100)
        self.publish_thread = Thread(target = self.manage_torques, name = 'Joseph')
        time.sleep(0.2)
        self.publish_thread.start()

    def update_internal_variables(self,data):
        "All these quantities are theoretically already low-pass filtered"
        with self.read_lock:
#             print('updating!')
            self.current_position = data.position
            self.current_velocity = data.velocity
            self.current_effort = data.effort
    def manage_torques(self):
        while True:
            with self.read_lock:
                with self.target_lock:
                    error = self.pos_target - self.current_position
                    vel = self.current_velocity
            self.error_integral += error
            self.error_integral = np.clip(self.error_integral,-self.i_saturation,self.i_saturation)
            self.command = self.conversion_constant*(self.k_p*error-self.k_d*vel+self.k_i*self.error_integral)
            msg = JointState()
            msg.effort = self.command
            self.joint_cmd_publisher.publish(msg)
            self.r.sleep()
            
        
    def set_target(self,pos_target):
        if(len(pos_target) != 16):
            print('wrong size config')
        else:
            pos_target = np.clip(pos_target,a_min = self.pos_min,a_max = self.pos_max)
            print(pos_target)
            with self.target_lock:
                self.pos_target = pos_target
        
        

In [3]:
hc = HandController()

In [29]:
tg = hc.pos_target
# tg[1] += 0.1
tg = [0.028027936105028343, 0.4334318352473945, 1.3604946093966854, 1.0028984983547073, 0.006796818909250197, -0.139290547459729, 0.7954028490304312, 0.728170085713102, 0.0468452962407659, -0.04385274957197116, 0.9093132355147681, 0.7231123110732285, 1.024910711807361, 0.3116512998229869, 0.6836375680440724, 0.9653240875116325]

hc.set_target(tg)

[0.02802794 0.43343184 1.36049461 1.0028985  0.00679682 0.
 0.79540285 0.72817009 0.0468453  0.         0.90931324 0.72311231
 1.02491071 0.3116513  0.68363757 0.96532409]


In [30]:
tg

[0.028027936105028343,
 0.4334318352473945,
 1.3604946093966854,
 1.0028984983547073,
 0.006796818909250197,
 -0.139290547459729,
 0.7954028490304312,
 0.728170085713102,
 0.0468452962407659,
 -0.04385274957197116,
 0.9093132355147681,
 0.7231123110732285,
 1.024910711807361,
 0.3116512998229869,
 0.6836375680440724,
 0.9653240875116325]

In [ ]:
# help(JointState)
state = JointState()

goal_position = [0.028027936105028343, 0.4334318352473945, 1.3604946093966854, 1.0028984983547073, 0.006796818909250197, -0.139290547459729, 0.7954028490304312, 0.728170085713102, 0.0468452962407659, -0.04385274957197116, 0.9093132355147681, 0.7231123110732285, 1.024910711807361, 0.3116512998229869, 0.6836375680440724, 0.9653240875116325]

effort = np.array([0.514141875905205083, 0.13133868394934473, -0.006045776600691687, -0.008459891164765511, 0.015981046195875315, 0.03172464593705451, -0.011637809953868334, -0.01849752361659694, 0.02030254564537881, 0.014990607686141195, -0.028567198016940857, -0.02527880594487169, -0.3104367712569881, -0.01792013245291449, 0.13417891663132614, 0.042600930938842145])

effort[:] = 0
effort[1] = 0

In [ ]:
# state.position = [-0.03277025449200767, 0.003013571106191393, 0.006906383669412847, 1.1, -0.010217954488392744, 0.004764243864069904, 0.0010231640262854237, 0.003937912161014311, 0.04439321845272195, 0.008805573227113736, -0.01304334889322642, 0.00889665153955775, 0.2509741202665393, 0.024889682055115662, -0.10084521357662637, -0.07196197865546952]
state.effort = effort

# import rospy
# from std_msgs.msg import String

test_pub = rospy.Publisher('/allegroHand_0/joint_cmd', JointState, queue_size=10)
rospy.init_node('TestNode')
# r = rospy.Rate(10) # 10hz
# while not rospy.is_shutdown():
#    pub.publish("hello world")
#    r.sleep()

test_pub.publish(state)

# Loading the URDF

In [ ]:
import pandas as pd
import klampt
from klampt import WorldModel
from klampt.model import ik
from klampt import vis
from copy import deepcopy
import numpy as np
from klampt.math import se3,so3

import time
vis.init('PyQt')

active_dofs = [6,7,8,9,11,12,13,14,16,17,18,19,21,22,23,24]

In [ ]:
world.loadElement('/home/motion/will/ROS_ALLEGRO_HAND/allegro_hand_ros_v4/src/allegro_hand_description/klampt_allegro_hand_right.urdf')

In [ ]:
world = WorldModel()

In [ ]:
vis.add('world',world)

vis.show()

In [ ]:
robot = world.robot(0)
zero_config = robot.getConfig()

In [ ]:
for link in range(robot.numLinks()):
    l = robot.link(link)
#     print(l.name)
orig_config = robot.getConfig()


In [ ]:
robot.setConfig(orig_config)

In [ ]:
# a = list(robot.getConfig())
a = deepcopy(orig_config)
a[21] += 1
robot.setConfig(a)

In [ ]:
len(robot.getConfig())

In [ ]:
hand_df = pd.read_pickle('jingchen_hand_run.pkl')
print(hand_df.shape)
# hand_df = hand_df.loc[254:780,:].reset_index(drop = True)
print(hand_df.shape)
pos = hand_df.positions[0]

max_vals = np.zeros(shape = (4,4))
max_vals[:] = -np.inf
min_vals = np.zeros(shape = (4,4))
min_vals[:] = np.inf
for angle in hand_df.angles:
    angle = angle.astype(int)
    max_vals = np.maximum(max_vals,angle)
    min_vals = np.minimum(min_vals,angle)
    
span = max_vals-min_vals
span

In [ ]:
toe_tip = (pos[0][3]-pos[0][0])/1000
toe_tip
pc = klampt.PointCloud()
# for i in range()
from klampt.math import se3,so3
R = so3.from_rotation_vector([0,-np.pi/2+0.25,0])
T = (R,[0,0,-0.2])

# pos = pos.reshape(-1,3)
# for pos in hand_df.positions[100:102]:
#     pc.setPoints(1.4*pos.reshape(-1,3)/1000)
#     pc.transform(R,[-0.04,-0.01,-0.05])
#     points = pc.getPoints()
#     unit_points =
# #     pc.transform(so3.identity(),t)
#     pc
#     vis.add('pc',pc,size = 40,color = [1,0,0,0.5])
#     time.sleep(0.2)


In [ ]:
active_dofs_list = [[11,12,13,14],[6,7,8,9],[16,17,18,19],[21,22,23,24]]
to_state_msg = [6,7,8,9,16,17,18,19,21,22,23,24,11,12,13,14]

In [ ]:
pc = klampt.PointCloud()
for pos,angle in zip(hand_df.positions,hand_df.angles):
    angle = angle.astype(float)
    pc.setPoints(1.4*pos.reshape(-1,3)/1000)
    pc.transform(R,[-0.04,-0.01,-0.05])
    vis.add('pc',pc,size = 40,color = [1,0,0,0.5])
    command = (np.pi/2)*(angle-min_vals)/span
    points = pc.getPoints()
#     robot.setConfig(orig_config)
    prev_config = robot.getConfig()
    toe = robot.link('link_15.0_tip')
    index = robot.link('link_3.0_tip')
    middle = robot.link('link_7.0_tip')
    ring = robot.link('link_11.0_tip')
    objs = []
    obj_toe = ik.objective(toe,local=[0,0,0.017],world=points[3])
#     obj_index = ik.objective(index,local=[0,0,0.017],world=points[7])
#     obj_middle = ik.objective(middle,local=[0,0,0.017],world=points[11])
#     obj_ring = ik.objective(ring,local=[0,0,0.017],world=points[19])
    iks_to_solve = {'toe':[obj_toe,[11,12,13,14]]}#,'index':[obj_index,[6,7,8,9]],
                    #'middle':[obj_middle,[16,17,18,19]],'ring':[obj_ring,[21,22,23,24]]}
#     objs = [obj_toe,obj_index,obj_middle,obj_ring]
    for key in iks_to_solve.keys():
        obj,active_dofs = iks_to_solve[key]
        prev_config = robot.getConfig()
        vis.lock()
        res = ik.solve_global(obj,activeDofs= active_dofs,numRestarts = 10)
        new_config = robot.getConfig()
        robot.setConfig(prev_config)
        if(res):
            robot.setConfig(new_config)
        vis.unlock()
    conf = robot.getConfig()
    conf[7:10] = command[0][1:]
    conf[17:20] = command[1][1:]
    conf[22:25] = command[2][1:]
    conf[21] = 0
    vis.lock()
    robot.setConfig(conf)
    vis.unlock()
    state = JointState()
    state.position = np.array(conf)[[to_state_msg]]
    test_pub.publish(state)
#     a = input()
    
    
    
    
#     print
    #     print(res)
    time.sleep(0.05)
# robot.setConfig(prev_config)


In [ ]:
print(new_config)

In [ ]:
solver.getResidual()

In [ ]:
hand_df

In [ ]:
finger_1.getTransform()

In [ ]:
len(new_config)

In [ ]:
help(vis.add)

# Testing HandController

In [ ]:
from allegro_control_demo import HandController, UDP_Server

# import pandas as pd
# import time
# hc = HandController()

# hand_df = pd.read_pickle('jingchen_hand_run.pkl')
# for pos,angle in zip(hand_df.positions,hand_df.angles):
#     hc.animate_and_command(pos,angle)
#     time.sleep(0.033)

server = UDP_Server(record = False,control = False)

In [ ]:
server.switch_control()